## Chat Models

### Langchain has two Entities
    1. Chat Models
    2. LLMs : They simply take text as input and produce text as output

In [30]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv(override=True)

True

In [31]:
model = ChatGroq(name='deepseek-r1-distill-qwen-32b',temperature=0,api_key=os.environ.get('GROQ_API_KEY'))

In [32]:
response = model.invoke("The Sky is")

In [33]:
print(response.content)

The sky is a vast and User, it is a natural wonder that we see every day. It is the atmosphere that surrounds the Earth, and it is filled with air, water vapor, and other gases. The sky appears blue during the day because of a process called Rayleigh scattering, which scatters the shorter-wavelength blue and violet light to a far greater degree than than longer-wavelength red, yellow, and green. However, we see the sky as blue, not violet, because our eyes are more sensitive to blue light and because sunlight reaches us more strongly in the blue part of the spectrum.

At night, the sky can be filled with stars, planets, and other celestial bodies. The appearance of the night sky can be affected by the presence of the Moon, as well as by clouds and other weather phenomena. In some locations, the sky can also be illuminated by artificial light, such as from cities and towns.

The sky is an important part of our environment, and it has been the subject of many poems, paintings, and other 

### Types of Message
1. SystemMessage : enables you to preconfigure your AI application to respond in a relatively predictable manner based on the user’s input.
2. HumanMessage : A message sent from the perspective of the human, with the user role
3. AIMessage : A message sent from the perspective of the AI that the human is interacting with, with the assistant role
4. ChatMessage : A message allowing for arbitrary setting of role

In [18]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ChatMessage

PROMPT = HumanMessage("What is the Capital of West Bengal?")
PROMPT

HumanMessage(content='What is the Capital of West Bengal?', additional_kwargs={}, response_metadata={})

In [15]:
model.invoke(PROMPT)

AIMessage(content="The capital of West Bengal, a state in eastern India, is divided into two cities: Kolkata (the former Calcutta) serves as the capital for administrative purposes, while the city of Bhubaneswar is the legislative capital where the state legislative assembly meets. Kolkata is the cultural, commercial, and educational hub of East India and is one of the country's most important port cities.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 16, 'total_tokens': 103, 'completion_time': 0.13300446, 'prompt_time': 0.002123382, 'queue_time': 0.019891576, 'total_time': 0.135127842}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-44b88571-e859-4ac4-b9c6-acbafe36ad3b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 87, 'total_tokens': 103})

In [12]:
PROMPT = [HumanMessage("What is the Capital of Odisha?")]

In [13]:
model.invoke(PROMPT)

AIMessage(content='The capital of Odisha, a state on the east coast of India, is Bhubaneswar. It is a major commercial, educational, and cultural hub and is known for its ancient temples and historical sites. Bhubaneswar, along with the nearby cities of Cuttack and Puri, forms the Golden Triangle, a popular tourist circuit in the state.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 15, 'total_tokens': 92, 'completion_time': 0.118067909, 'prompt_time': 0.001992863, 'queue_time': 0.020265986, 'total_time': 0.120060772}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-3c604384-bd9d-4716-b772-a5e5e9b4be68-0', usage_metadata={'input_tokens': 15, 'output_tokens': 77, 'total_tokens': 92})

In [19]:
SYSTEM = SystemMessage(''' You are a helpful assistant equipped with latest information , you will answer use query and end your answer with JAHAPANA. Always''')

In [20]:
model.invoke([SYSTEM, PROMPT])

AIMessage(content='The capital of West Bengal is Kolkata. JAHAPANA.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 44, 'total_tokens': 62, 'completion_time': 0.026584742, 'prompt_time': 0.005142366, 'queue_time': 0.019643003000000003, 'total_time': 0.031727108}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e15966f-7578-470b-a54b-ca5377a767d3-0', usage_metadata={'input_tokens': 44, 'output_tokens': 18, 'total_tokens': 62})

### Structured Outputs

In [5]:
from pydantic import BaseModel, Field
class AnswerSchema(BaseModel):
    ''' Class defining the structure we want the final response from LLM to be'''
    answer: str = Field('Simple, Straight answer to the question asked')
    justification : str= Field('Detailed breakdown which supports the answer and make sense to user')

In [6]:
structured_model = model.with_structured_output(AnswerSchema)
structured_model.invoke("What weighs more, a pound of bricks or a pound of feathers")

AnswerSchema(answer='A pound of bricks and a pound of feathers both weigh the same, which is one pound.', justification='The weight of an object is determined by its mass, and a pound is a unit of mass. Therefore, a pound of any material will always weigh the same.')

### The Runnable Interface

    invoke() takes a single input and returns a single output.

    batch() takes a list of outputs and returns a list of outputs.
    
    stream() takes a single input and returns an iterator of parts of the output as they become available.

#### RunnablePassThrough
    Utility designed to streamline data processing workflows by either passing input data unchanged or enhancing it with additional attributes

    1. RunnablePassthrough can either pass the input unchanged or append additional keys to it.
    2. When RunnablePassthrough() is called on its own, it simply takes the input and passes it as is.
    3. When called using RunnablePassthrough.assign(...), it takes the input and adds additional arguments provided to the assign function.

    Use Cases

    1. Useful when you need to pass data through chain steps without modification
    2. Can be combined with other components to build complex data pipelines
    3. Particularly helpful when you need to preserve original input while adding new fields

In [14]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.prompts import PromptTemplate

PROMPT_EX = PromptTemplate.from_template("What is 10 times {num}")
chain = {'num': RunnablePassthrough()} | RunnableLambda(lambda x : x) | PROMPT_EX | structured_model

chain.invoke(9)

AnswerSchema(answer='90', justification='The result of multiplying 10 by 9')

In [15]:
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

### RunnableParallel 
    Utility designed to execute multiple Runnable objects concurrently, streamlining workflows that require parallel processing. 
    It distributes input data across different components, collects their results, and combines them into a unified output. 
    This functionality makes it a powerful tool for optimizing workflows where tasks can be performed independently and simultaneously.

In [17]:
#Chains can also be applied to RunnableParallel.

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("What is the capital of {country}?")
    | structured_model
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("What is the area of {country}?")
    | structured_model
)

combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("United States of America")

{'capital': AnswerSchema(answer='The capital of the United States of America is Washington, D.C.', justification='Washington, D.C. is specifically designated as the capital of the United States by the United States Constitution.'),
 'area': AnswerSchema(answer='The area of the United States of America is approximately 9.83 million square kilometers.', justification='The United States of America is the third largest country by area, covering a significant portion of the North American continent. Its total area, including both land and water, is approximately 9.83 million square kilometers.')}

### RunnableLambda
    RunnableLambda is a flexible utility that allows developers to define custom data transformation logic using lambda functions. 
    By enabling quick and easy implementation of custom processing workflows, RunnableLambda simplifies the creation of tailored data pipelines 
    while ensuring minimal setup overhead.

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from datetime import datetime

def get_today(a):
    # Get today's date
    return 'Nov 19'

# Print today's date
get_today(None)

'Nov 19'

In [25]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Create the prompt and llm
prompt = PromptTemplate.from_template(
    "List {n} famous people from India whose birthday is on {today}. Include their date of birth."
)

# Create the chain
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [26]:
print(chain.invoke(3))

1. Indira Gandhi: She was the first and, to date, the only female Prime Minister of India. She was born on November 19, 1917. Indira Gandhi served as Prime Minister from 1966 to 1977 and then again from 1980 until her assassination in 1984.

2. Prakash Padukone: He is a former Indian badminton player who was ranked World No. 1 in 1980. Born on November 19, 1955, Padukone was the first Indian to win the All England Open Badminton Championships in 1980.

3. Sarojini Naidu: She was a prominent Indian independence activist and poet. Sarojini Naidu was born on November 19, 1879, and became the first Indian woman to become the President of the Indian National Congress and the first woman to hold the office of Governor, serving as the Governor of Uttar Pradesh from 1947 to 1949.


### itemgetter
    itemgetter is a utility function from Python's operator module with the following features and benefits:
    
    Core Functionality
    
    Efficiently extracts values from specific keys or indices in dictionaries, tuples, and lists
    Capable of extracting multiple keys or indices simultaneously
    Supports functional programming style
    Performance Optimization
    
    More efficient than regular indexing for repetitive key access operations
    Optimized memory usage
    Performance advantages when processing large datasets
    Usage in LangChain
    
    Data filtering in chain compositions
    Selective extraction from complex input structures
    Combines with other Runnable objects for data preprocessing

In [28]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda



# Function that returns the length of a sentence.
def length_function(text):
    return len(text)


# Function that returns the product of the lengths of two sentences.
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# Function that uses _multiple_length_function to return the product of the lengths of two sentences.
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("What is {a} + {b}?")

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

In [29]:
chain.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='The sum of 5 and 25 is 30. You can calculate this by adding the two numbers together:\n\n5 + 25 = 30\n\nThis is a basic arithmetic operation called addition. It combines two numbers to produce a third number, which is the sum of the original two numbers. In this case, the sum of 5 and 25 is 30.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 16, 'total_tokens': 105, 'completion_time': 0.136137326, 'prompt_time': 0.002681811, 'queue_time': 0.019197168, 'total_time': 0.138819137}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-dcbfa997-5128-4ea9-901c-c75d60c99dc2-0', usage_metadata={'input_tokens': 16, 'output_tokens': 89, 'total_tokens': 105})